# Banking Dataset Lead Scoring

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt  #Visualization
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.columns

Index(['ID', 'Gender', 'DOB', 'Lead_Creation_Date', 'City_Code',
       'City_Category', 'Employer_Code', 'Employer_Category1',
       'Employer_Category2', 'Monthly_Income',
       'Customer_Existing_Primary_Bank_Code', 'Primary_Bank_Type', 'Contacted',
       'Source', 'Source_Category', 'Existing_EMI', 'Loan_Amount',
       'Loan_Period', 'Interest_Rate', 'EMI', 'Var1', 'Approved'],
      dtype='object')

In [4]:
df.describe()

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
count,65415.000000,6.971300e+04,69662.000000,42004.000000,42004.000000,22276.000000,22276.000000,69713.000000,69713.000000
mean,3.720187,5.622283e+03,360.928751,39429.982859,3.890629,19.213570,1101.466242,3.948446,0.014631
std,0.807374,1.747671e+05,2288.517927,30727.595990,1.167491,5.847136,752.661394,3.819214,0.120073
min,1.000000,0.000000e+00,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000,0.000000
25%,4.000000,1.650000e+03,0.000000,20000.000000,3.000000,15.250000,649.000000,0.000000,0.000000
50%,4.000000,2.500000e+03,0.000000,30000.000000,4.000000,18.000000,941.000000,2.000000,0.000000
75%,4.000000,4.000000e+03,350.000000,50000.000000,5.000000,20.000000,1295.000000,7.000000,0.000000
max,4.000000,3.838384e+07,545436.500000,300000.000000,6.000000,37.000000,13556.000000,10.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69713 entries, 0 to 69712
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ID                                   69713 non-null  object 
 1   Gender                               69713 non-null  object 
 2   DOB                                  69698 non-null  object 
 3   Lead_Creation_Date                   69713 non-null  object 
 4   City_Code                            68899 non-null  object 
 5   City_Category                        68899 non-null  object 
 6   Employer_Code                        65695 non-null  object 
 7   Employer_Category1                   65695 non-null  object 
 8   Employer_Category2                   65415 non-null  float64
 9   Monthly_Income                       69713 non-null  float64
 10  Customer_Existing_Primary_Bank_Code  60322 non-null  object 
 11  Primary_Bank_Type           

In [6]:
df.isnull().sum()

ID                                         0
Gender                                     0
DOB                                       15
Lead_Creation_Date                         0
City_Code                                814
City_Category                            814
Employer_Code                           4018
Employer_Category1                      4018
Employer_Category2                      4298
Monthly_Income                             0
Customer_Existing_Primary_Bank_Code     9391
Primary_Bank_Type                       9391
Contacted                                  0
Source                                     0
Source_Category                            0
Existing_EMI                              51
Loan_Amount                            27709
Loan_Period                            27709
Interest_Rate                          47437
EMI                                    47437
Var1                                       0
Approved                                   0
dtype: int

In [7]:
nullCheck=pd.DataFrame()
nullCheck['Number of null values']=df.isnull().sum()
nullCheck['Percentage of Null Values']=(df.isnull().sum() / df.shape[0]) * 100
nullCheck=nullCheck.sort_values('Percentage of Null Values',ascending=False)
nullCheck


,Number of null values,Percentage of Null Values
EMI,47437,68.046132
Interest_Rate,47437,68.046132
Loan_Period,27709,39.747249
Loan_Amount,27709,39.747249
Primary_Bank_Type,9391,13.470945
Customer_Existing_Primary_Bank_Code,9391,13.470945
Employer_Category2,4298,6.165278
Employer_Code,4018,5.763631
Employer_Category1,4018,5.763631
City_Code,814,1.167644


loan amount and loan period = 0 for null values

In [8]:
df['Loan_Amount']=df['Loan_Amount'].fillna(0)
df['Loan_Period']=df['Loan_Period'].fillna(0)

# Extracting Data to predict unknown interest rates 

In [9]:
df2=df.iloc[:,7:10]
df3=df.iloc[ :,16:19] #loan period
df2=pd.concat([df2,df3],axis='columns')

df3=df2[df2['Interest_Rate'].isnull()==False]
df3=df3.dropna()

In [10]:
Y=df3['Interest_Rate']
X=df3.drop(['Interest_Rate'],axis='columns')

In [11]:
X

,Employer_Category1,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period
1,C,1.0,3500.0,20000.0,2.0
6,B,4.0,7500.0,130000.0,5.0
7,A,4.0,3000.0,30000.0,3.0
8,A,4.0,2500.0,66000.0,5.0
10,B,4.0,2700.0,20000.0,5.0
...,...,...,...,...,...
69703,A,4.0,3500.0,87000.0,5.0
69704,B,4.0,13300.0,20000.0,4.0
69706,B,2.0,4600.0,30000.0,3.0
69710,C,4.0,1600.0,24000.0,4.0


In [12]:
Y

1        13.25
6        14.85
7        18.25
8        20.00
10       18.00
         ...  
69703    15.25
69704    13.99
69706    13.00
69710    35.50
69712    13.99
Name: Interest_Rate, Length: 22104, dtype: float64

In [13]:
# X=X.dropna()

In [14]:
dummies1=pd.get_dummies(X['Employer_Category1'])
dummies1=dummies1.drop(dummies1.columns[0],axis='columns')
X=X.drop(['Employer_Category1'],axis='columns')
X=pd.concat([X,dummies1],axis='columns')

In [15]:
X

,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,B,C
1,1.0,3500.0,20000.0,2.0,0,1
6,4.0,7500.0,130000.0,5.0,1,0
7,4.0,3000.0,30000.0,3.0,0,0
8,4.0,2500.0,66000.0,5.0,0,0
10,4.0,2700.0,20000.0,5.0,1,0
...,...,...,...,...,...,...
69703,4.0,3500.0,87000.0,5.0,0,0
69704,4.0,13300.0,20000.0,4.0,1,0
69706,2.0,4600.0,30000.0,3.0,1,0
69710,4.0,1600.0,24000.0,4.0,0,1


In [16]:
from xgboost import XGBRegressor
xgbr=XGBRegressor()

In [17]:
xgbr.fit(X,Y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [18]:
df4=df2[df2['Interest_Rate'].isnull()==True]
df5=df4[df4['Loan_Amount']!=0]

In [19]:
df5

,Employer_Category1,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,Interest_Rate
2,C,4.0,2250.0,45000.0,4.0,NaN
3,A,4.0,3500.0,92000.0,5.0,NaN
4,A,4.0,10000.0,50000.0,2.0,NaN
13,B,4.0,3150.0,45000.0,5.0,NaN
14,B,1.0,6000.0,104000.0,5.0,NaN
...,...,...,...,...,...,...
69698,A,4.0,3200.0,50000.0,5.0,NaN
69702,A,4.0,4000.0,57000.0,4.0,NaN
69705,A,4.0,1500.0,24000.0,4.0,NaN
69707,A,1.0,2400.0,30000.0,3.0,NaN


In [20]:
df2

,Employer_Category1,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,Interest_Rate
0,A,4.0,2000.0,0.0,0.0,NaN
1,C,1.0,3500.0,20000.0,2.0,13.25
2,C,4.0,2250.0,45000.0,4.0,NaN
3,A,4.0,3500.0,92000.0,5.0,NaN
4,A,4.0,10000.0,50000.0,2.0,NaN
...,...,...,...,...,...,...
69708,A,1.0,4900.0,0.0,0.0,NaN
69709,A,4.0,7190.1,0.0,0.0,NaN
69710,C,4.0,1600.0,24000.0,4.0,35.50
69711,C,3.0,9893.0,80000.0,5.0,NaN


In [21]:
X_test=df5.drop(['Interest_Rate'],axis='columns')

dummies2=pd.get_dummies(X_test['Employer_Category1'])
dummies2=dummies2.drop(dummies2.columns[0],axis='columns')
X_test=X_test.drop(['Employer_Category1'],axis='columns')
X_test=pd.concat([X_test,dummies2],axis='columns')

In [22]:
X_test

,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,B,C
2,4.0,2250.0,45000.0,4.0,0,1
3,4.0,3500.0,92000.0,5.0,0,0
4,4.0,10000.0,50000.0,2.0,0,0
13,4.0,3150.0,45000.0,5.0,1,0
14,1.0,6000.0,104000.0,5.0,1,0
...,...,...,...,...,...,...
69698,4.0,3200.0,50000.0,5.0,0,0
69702,4.0,4000.0,57000.0,4.0,0,0
69705,4.0,1500.0,24000.0,4.0,0,0
69707,1.0,2400.0,30000.0,3.0,0,0


In [23]:
Y_predicted=xgbr.predict(X_test)

In [24]:
df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,...,N,S122,G,0.0,0.0,0.0,NaN,NaN,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,...,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,...,Y,S143,B,0.0,45000.0,4.0,NaN,NaN,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,...,Y,S143,B,0.0,92000.0,5.0,NaN,NaN,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,...,Y,S134,B,2500.0,50000.0,2.0,NaN,NaN,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69708,APPU90955789628,Female,31/07/83,30/09/16,C10006,A,COM0000010,A,1.0,4900.0,...,N,S122,G,0.0,0.0,0.0,NaN,NaN,10,0
69709,APPV80989824738,Female,27/01/71,30/09/16,C10116,C,COM0045789,A,4.0,7190.1,...,N,S122,G,1450.0,0.0,0.0,NaN,NaN,7,0
69710,APPW50697209842,Female,01/02/92,30/09/16,C10022,B,COM0013284,C,4.0,1600.0,...,Y,S122,G,0.0,24000.0,4.0,35.50,943.0,2,0
69711,APPY50870035036,Male,27/06/78,30/09/16,C10002,A,COM0000098,C,3.0,9893.0,...,Y,S122,G,1366.0,80000.0,5.0,NaN,NaN,10,0


In [25]:
df['Interest_Rate']=df['Interest_Rate'].fillna(-1)
df['EMI']=df['EMI'].fillna(-1)

In [26]:
def fill_ir(df,Y_predicted,x):
    for i in range(df.shape[0]):  #16 and 18
        la=df.iloc[i,16]
        #df.iloc[i,18]   ir
        if(la==0):
          df.iloc[i,18]=0    
        elif ((la!=0) and (df.iloc[i,18]==-1)):
          df.iloc[i,18]=Y_predicted[x]
          x+=1
    df['Interest_Rate']=df['Interest_Rate'].round(1)



In [27]:
fill_ir(df,Y_predicted,0)

In [28]:
df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,...,N,S122,G,0.0,0.0,0.0,0.0,-1.0,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,...,Y,S122,G,0.0,20000.0,2.0,13.2,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,...,Y,S143,B,0.0,45000.0,4.0,18.5,-1.0,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,...,Y,S143,B,0.0,92000.0,5.0,19.4,-1.0,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,...,Y,S134,B,2500.0,50000.0,2.0,15.2,-1.0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69708,APPU90955789628,Female,31/07/83,30/09/16,C10006,A,COM0000010,A,1.0,4900.0,...,N,S122,G,0.0,0.0,0.0,0.0,-1.0,10,0
69709,APPV80989824738,Female,27/01/71,30/09/16,C10116,C,COM0045789,A,4.0,7190.1,...,N,S122,G,1450.0,0.0,0.0,0.0,-1.0,7,0
69710,APPW50697209842,Female,01/02/92,30/09/16,C10022,B,COM0013284,C,4.0,1600.0,...,Y,S122,G,0.0,24000.0,4.0,35.5,943.0,2,0
69711,APPY50870035036,Male,27/06/78,30/09/16,C10002,A,COM0000098,C,3.0,9893.0,...,Y,S122,G,1366.0,80000.0,5.0,14.4,-1.0,10,0


In [29]:
def fill_emi(df):
    for i in range(df.shape[0]):
        la=df.iloc[i,16]
        if(la==0):
            df.iloc[i,19]=0
        elif(df.iloc[i,19]==-1):
            total=la+((la*df.iloc[i,19])/100) #emi formula
            emi=total/(12*df.iloc[i,17])
            df.iloc[i,19]=emi
    df['EMI']=df['EMI'].round(1)
            

In [30]:
fill_emi(df)

In [31]:
df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,...,N,S122,G,0.0,0.0,0.0,0.0,0.0,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,...,Y,S122,G,0.0,20000.0,2.0,13.2,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,...,Y,S143,B,0.0,45000.0,4.0,18.5,928.1,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,...,Y,S143,B,0.0,92000.0,5.0,19.4,1518.0,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,...,Y,S134,B,2500.0,50000.0,2.0,15.2,2062.5,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69708,APPU90955789628,Female,31/07/83,30/09/16,C10006,A,COM0000010,A,1.0,4900.0,...,N,S122,G,0.0,0.0,0.0,0.0,0.0,10,0
69709,APPV80989824738,Female,27/01/71,30/09/16,C10116,C,COM0045789,A,4.0,7190.1,...,N,S122,G,1450.0,0.0,0.0,0.0,0.0,7,0
69710,APPW50697209842,Female,01/02/92,30/09/16,C10022,B,COM0013284,C,4.0,1600.0,...,Y,S122,G,0.0,24000.0,4.0,35.5,943.0,2,0
69711,APPY50870035036,Male,27/06/78,30/09/16,C10002,A,COM0000098,C,3.0,9893.0,...,Y,S122,G,1366.0,80000.0,5.0,14.4,1320.0,10,0


In [32]:
nullCheck=pd.DataFrame()
nullCheck['Number of null values']=df.isnull().sum()
nullCheck['Percentage of Null Values']=(df.isnull().sum() / df.shape[0]) * 100
nullCheck=nullCheck.sort_values('Percentage of Null Values',ascending=False)
nullCheck

,Number of null values,Percentage of Null Values
Primary_Bank_Type,9391,13.470945
Customer_Existing_Primary_Bank_Code,9391,13.470945
Employer_Category2,4298,6.165278
Employer_Code,4018,5.763631
Employer_Category1,4018,5.763631
City_Code,814,1.167644
City_Category,814,1.167644
Existing_EMI,51,0.073157
DOB,15,0.021517
Var1,0,0.000000


In [33]:
df[df.duplicated()==True]

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved


In [34]:
df=df.dropna()

In [35]:
df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
0,APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,...,N,S122,G,0.0,0.0,0.0,0.0,0.0,0,0
1,APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,...,Y,S122,G,0.0,20000.0,2.0,13.2,953.0,10,0
2,APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,...,Y,S143,B,0.0,45000.0,4.0,18.5,928.1,0,0
3,APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,...,Y,S143,B,0.0,92000.0,5.0,19.4,1518.0,7,0
4,APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,...,Y,S134,B,2500.0,50000.0,2.0,15.2,2062.5,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69707,APPT50870248519,Male,03/03/91,30/09/16,C10041,C,COM0000009,A,1.0,2400.0,...,Y,S122,G,0.0,30000.0,3.0,18.9,825.0,2,0
69708,APPU90955789628,Female,31/07/83,30/09/16,C10006,A,COM0000010,A,1.0,4900.0,...,N,S122,G,0.0,0.0,0.0,0.0,0.0,10,0
69709,APPV80989824738,Female,27/01/71,30/09/16,C10116,C,COM0045789,A,4.0,7190.1,...,N,S122,G,1450.0,0.0,0.0,0.0,0.0,7,0
69710,APPW50697209842,Female,01/02/92,30/09/16,C10022,B,COM0013284,C,4.0,1600.0,...,Y,S122,G,0.0,24000.0,4.0,35.5,943.0,2,0


In [36]:
df=df.drop(['Employer_Code','ID','City_Code','Customer_Existing_Primary_Bank_Code','Var1',
            'Source','Employer_Category1'],axis='columns')

In [37]:
df=df.reset_index(drop=True)

In [38]:
df

,Gender,DOB,Lead_Creation_Date,City_Category,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Approved
0,Female,23/07/79,15/07/16,A,4.0,2000.0,P,N,G,0.0,0.0,0.0,0.0,0.0,0
1,Male,07/12/86,04/07/16,A,1.0,3500.0,P,Y,G,0.0,20000.0,2.0,13.2,953.0,0
2,Male,10/12/82,19/07/16,C,4.0,2250.0,G,Y,B,0.0,45000.0,4.0,18.5,928.1,0
3,Male,30/01/89,09/07/16,C,4.0,3500.0,G,Y,B,0.0,92000.0,5.0,19.4,1518.0,0
4,Male,19/04/85,20/07/16,A,4.0,10000.0,P,Y,B,2500.0,50000.0,2.0,15.2,2062.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59541,Male,03/03/91,30/09/16,C,1.0,2400.0,G,Y,G,0.0,30000.0,3.0,18.9,825.0,0
59542,Female,31/07/83,30/09/16,A,1.0,4900.0,P,N,G,0.0,0.0,0.0,0.0,0.0,0
59543,Female,27/01/71,30/09/16,C,4.0,7190.1,P,N,G,1450.0,0.0,0.0,0.0,0.0,0
59544,Female,01/02/92,30/09/16,B,4.0,1600.0,P,Y,G,0.0,24000.0,4.0,35.5,943.0,0


In [39]:
df=df.drop(['DOB','Lead_Creation_Date'],axis='columns')

# Categorical Encoding

In [40]:
dictionary={ 'Gender':{'Male':1,'Female':0},
      'Contacted':{'Y':1,'N':0},
    'Primary_Bank_Type':{'P':1,'G':0}
}

df=df.replace(dictionary)

In [41]:
df.City_Category.unique()

array(['A', 'C', 'B'], dtype=object)

In [42]:
df.Source_Category.unique()

array(['G', 'B', 'C', 'E', 'F', 'A', 'D'], dtype=object)

In [43]:
dummies1=pd.get_dummies(df.City_Category)
# dummies1=dummies1.drop(dummies1.columns[0],axis='columns')
# X=pd.concat([X,dummies1],axis='columns')

dummies1.columns={'City_A','City_B','City_C'}

dummies1=dummies1.drop(dummies1.columns[0],axis='columns')
df=pd.concat([df,dummies1],axis='columns')

dummies2=pd.get_dummies(df.Source_Category)

dummies2=dummies2.drop(dummies2.columns[0],axis='columns')
df=pd.concat([df,dummies2],axis='columns')

In [44]:
df

,Gender,City_Category,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,...,EMI,Approved,City_C,City_A,B,C,D,E,F,G
0,0,A,4.0,2000.0,1,0,G,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,1,A,1.0,3500.0,1,1,G,0.0,20000.0,2.0,...,953.0,0,0,0,0,0,0,0,0,1
2,1,C,4.0,2250.0,0,1,B,0.0,45000.0,4.0,...,928.1,0,0,1,1,0,0,0,0,0
3,1,C,4.0,3500.0,0,1,B,0.0,92000.0,5.0,...,1518.0,0,0,1,1,0,0,0,0,0
4,1,A,4.0,10000.0,1,1,B,2500.0,50000.0,2.0,...,2062.5,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59541,1,C,1.0,2400.0,0,1,G,0.0,30000.0,3.0,...,825.0,0,0,1,0,0,0,0,0,1
59542,0,A,1.0,4900.0,1,0,G,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,1
59543,0,C,4.0,7190.1,1,0,G,1450.0,0.0,0.0,...,0.0,0,0,1,0,0,0,0,0,1
59544,0,B,4.0,1600.0,1,1,G,0.0,24000.0,4.0,...,943.0,0,1,0,0,0,0,0,0,1


In [45]:
df=df.drop(['City_Category','Source_Category'],axis='columns')

# Splitting Data

In [46]:
Y=df['Approved']
X=df.drop(['Approved'],axis='columns')

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,random_state=99)

In [48]:
X_train

,Gender,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,City_C,City_A,B,C,D,E,F,G
28913,0,4.0,3000.0,1,0,1550.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0,0
47379,1,4.0,2500.0,1,1,0.0,40000.0,4.0,14.8,1110.0,1,0,0,0,0,0,0,1
55707,1,4.0,2000.0,1,1,500.0,15000.0,3.0,33.0,662.0,0,1,0,0,0,0,0,1
936,1,4.0,2550.0,1,1,200.0,10000.0,3.0,16.8,275.0,0,0,1,0,0,0,0,0
48151,1,4.0,3500.0,1,1,0.0,60000.0,4.0,15.6,1237.5,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42697,0,4.0,1300.0,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
36008,1,2.0,2400.0,1,1,0.0,40000.0,4.0,16.8,1149.0,0,0,0,1,0,0,0,0
46265,1,4.0,1777.4,1,1,0.0,10000.0,4.0,24.6,206.2,0,1,0,0,0,0,0,1
23587,0,4.0,1600.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0


# Model Training

In [49]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [50]:
xgb.score(X_test,Y_test)

0.9830394626364399

In [51]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(xgb,X_train,Y_train,cv=5)
scores

array([0.98299748, 0.98310066, 0.9829957 , 0.9829957 , 0.98310066])

In [52]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X_train,Y_train)

SVC()

In [53]:
svc.score(X_test,Y_test)

0.9831234256926952

In [54]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(svc,X_train,Y_train,cv=5)
scores

array([0.9834173 , 0.98341556, 0.98341556, 0.98341556, 0.98341556])

# Metrics

In [55]:
xgb.fit(X,Y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [56]:
test_df=pd.read_csv('test.csv')

In [57]:
test_df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
0,APPA70109647212,Male,03/06/88,05/07/16,C10028,C,COM0002222,A,4.0,2150.0,...,P,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4
1,APPB10687939341,Male,13/07/81,01/07/16,C10003,A,COM0001784,C,4.0,4200.0,...,P,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7
2,APPC80449411414,Female,19/11/90,01/07/16,C10009,B,COM0045260,B,4.0,1000.0,...,P,N,S133,B,0.0,NaN,NaN,NaN,NaN,0
3,APPD30665094501,Female,15/10/92,01/07/16,C10005,A,COM0000085,A,3.0,1465.0,...,P,N,S133,C,0.0,NaN,NaN,NaN,NaN,0
4,APPE80379821637,Male,21/09/88,01/07/16,C10005,A,COM0006422,A,4.0,2340.0,...,P,Y,S143,B,500.0,10000.0,2.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30032,APPH10161458941,Male,19/11/80,30/09/16,C10004,A,COM0000490,C,4.0,1240.0,...,G,Y,S122,G,0.0,20000.0,4.0,31.5,738.0,2
30033,APPK90372617208,Male,23/08/93,30/09/16,C10014,B,COM0032434,A,4.0,2000.0,...,G,Y,S122,G,0.0,32000.0,4.0,NaN,NaN,2
30034,APPL30263849328,Female,01/07/90,30/09/16,C10002,A,COM0016561,A,4.0,1000.0,...,P,N,S122,G,350.0,NaN,NaN,NaN,NaN,0
30035,APPM70251572449,Female,27/02/73,30/09/16,C10177,C,COM0000351,A,4.0,4000.0,...,G,N,S122,G,1000.0,NaN,NaN,NaN,NaN,7


In [58]:
nullCheck=pd.DataFrame()
nullCheck['Number of null values']=test_df.isnull().sum()
nullCheck['Percentage of Null Values']=(test_df.isnull().sum() / test_df.shape[0]) * 100
nullCheck=nullCheck.sort_values('Percentage of Null Values',ascending=False)
nullCheck

,Number of null values,Percentage of Null Values
EMI,20385,67.866298
Interest_Rate,20385,67.866298
Loan_Period,11871,39.521257
Loan_Amount,11871,39.521257
Customer_Existing_Primary_Bank_Code,4037,13.440091
Primary_Bank_Type,4037,13.440091
Employer_Category2,1695,5.643040
Employer_Code,1605,5.343410
Employer_Category1,1605,5.343410
City_Code,314,1.045377


In [59]:
test_df['Loan_Amount']=test_df['Loan_Amount'].fillna(0)
test_df['Loan_Period']=test_df['Loan_Period'].fillna(0)

In [60]:
test_df2=test_df.iloc[:,7:10]
test_df3=test_df.iloc[ :,16:19] #loan period
test_df2=pd.concat([test_df2,test_df3],axis='columns')

In [61]:
test_df4=test_df2[test_df2['Interest_Rate'].isnull()==True]
test_df5=test_df4[test_df4['Loan_Amount']!=0]

In [62]:
test_df5

,Employer_Category1,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,Interest_Rate
4,A,4.0,2340.0,10000.0,2.0,NaN
8,A,4.0,3000.0,10000.0,1.0,NaN
9,B,4.0,4040.0,30000.0,4.0,NaN
15,B,4.0,3000.0,30000.0,3.0,NaN
19,A,4.0,2000.0,10000.0,2.0,NaN
...,...,...,...,...,...,...
30011,B,4.0,7900.0,50000.0,3.0,NaN
30020,A,4.0,1600.0,10000.0,3.0,NaN
30023,A,4.0,3000.0,55000.0,4.0,NaN
30027,A,4.0,2100.0,49000.0,5.0,NaN


In [63]:
X_test=test_df5.drop(['Interest_Rate'],axis='columns')

dummies2=pd.get_dummies(X_test['Employer_Category1'])
dummies2=dummies2.drop(dummies2.columns[0],axis='columns')
X_test=X_test.drop(['Employer_Category1'],axis='columns')
X_test=pd.concat([X_test,dummies2],axis='columns')

In [64]:
X_test

,Employer_Category2,Monthly_Income,Loan_Amount,Loan_Period,B,C
4,4.0,2340.0,10000.0,2.0,0,0
8,4.0,3000.0,10000.0,1.0,0,0
9,4.0,4040.0,30000.0,4.0,1,0
15,4.0,3000.0,30000.0,3.0,1,0
19,4.0,2000.0,10000.0,2.0,0,0
...,...,...,...,...,...,...
30011,4.0,7900.0,50000.0,3.0,1,0
30020,4.0,1600.0,10000.0,3.0,0,0
30023,4.0,3000.0,55000.0,4.0,0,0
30027,4.0,2100.0,49000.0,5.0,0,0


In [65]:
Y_predicted=xgbr.predict(X_test)

In [66]:
test_df['Interest_Rate']=test_df['Interest_Rate'].fillna(-1)
test_df['EMI']=test_df['EMI'].fillna(-1)

In [67]:
fill_ir(test_df,Y_predicted,0)
fill_emi(test_df)

In [68]:
test_df=test_df.dropna()
final_df=test_df.copy()
test_df=test_df.drop(['Employer_Code','ID','City_Code','Customer_Existing_Primary_Bank_Code','Var1',
            'Source','Employer_Category1'],axis='columns')

In [69]:
test_df=test_df.drop(['DOB','Lead_Creation_Date'],axis='columns')
test_df=test_df.reset_index(drop=True)

In [70]:
test_df=test_df.replace(dictionary)
dummies1=pd.get_dummies(test_df.City_Category)
# dummies1=dummies1.drop(dummies1.columns[0],axis='columns')
# X=pd.concat([X,dummies1],axis='columns')

dummies1.columns={'City_A','City_B','City_C'}

dummies1=dummies1.drop(dummies1.columns[0],axis='columns')
test_df=pd.concat([test_df,dummies1],axis='columns')

dummies2=pd.get_dummies(test_df.Source_Category)

dummies2=dummies2.drop(dummies2.columns[0],axis='columns')
test_df=pd.concat([test_df,dummies2],axis='columns')

test_df=test_df.drop(['City_Category','Source_Category'],axis='columns')

In [71]:
test_df

,Gender,Employer_Category2,Monthly_Income,Primary_Bank_Type,Contacted,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,City_C,City_A,B,C,D,E,F,G
0,1,4.0,2150.0,1,1,0.0,10000.0,3.0,20.0,372.0,0,1,1,0,0,0,0,0
1,1,4.0,4200.0,1,1,0.0,69000.0,5.0,24.0,1985.0,0,0,0,1,0,0,0,0
2,0,4.0,1000.0,1,0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0,0,0,0
3,0,3.0,1465.0,1,0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0
4,1,4.0,2340.0,1,1,500.0,10000.0,2.0,24.0,412.5,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25702,1,4.0,1240.0,0,1,0.0,20000.0,4.0,31.5,738.0,0,0,0,0,0,0,0,1
25703,1,4.0,2000.0,0,1,0.0,32000.0,4.0,26.3,660.0,1,0,0,0,0,0,0,1
25704,0,4.0,1000.0,1,0,350.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1
25705,0,4.0,4000.0,0,0,1000.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,1


In [72]:
probs=xgb.predict_proba(test_df)

In [73]:
probs_array=[]
for i in probs:
    if i[0]>i[1]:
        probs_array.append((1-i[0])*100)
    else:
        probs_array.append((i[1])*100)


In [74]:
probs_array = ["%.2f" % x for x in probs_array]

# print(probs_array)

In [75]:
final_df['Scores']=probs_array

In [76]:
final_df

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Scores
0,APPA70109647212,Male,03/06/88,05/07/16,C10028,C,COM0002222,A,4.0,2150.0,...,Y,S122,B,0.0,10000.0,3.0,20.0,372.0,4,2.97
1,APPB10687939341,Male,13/07/81,01/07/16,C10003,A,COM0001784,C,4.0,4200.0,...,Y,S133,C,0.0,69000.0,5.0,24.0,1985.0,7,0.35
2,APPC80449411414,Female,19/11/90,01/07/16,C10009,B,COM0045260,B,4.0,1000.0,...,N,S133,B,0.0,0.0,0.0,0.0,0.0,0,0.00
3,APPD30665094501,Female,15/10/92,01/07/16,C10005,A,COM0000085,A,3.0,1465.0,...,N,S133,C,0.0,0.0,0.0,0.0,0.0,0,0.00
4,APPE80379821637,Male,21/09/88,01/07/16,C10005,A,COM0006422,A,4.0,2340.0,...,Y,S143,B,500.0,10000.0,2.0,24.0,412.5,0,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30032,APPH10161458941,Male,19/11/80,30/09/16,C10004,A,COM0000490,C,4.0,1240.0,...,Y,S122,G,0.0,20000.0,4.0,31.5,738.0,2,0.01
30033,APPK90372617208,Male,23/08/93,30/09/16,C10014,B,COM0032434,A,4.0,2000.0,...,Y,S122,G,0.0,32000.0,4.0,26.3,660.0,2,0.08
30034,APPL30263849328,Female,01/07/90,30/09/16,C10002,A,COM0016561,A,4.0,1000.0,...,N,S122,G,350.0,0.0,0.0,0.0,0.0,0,0.00
30035,APPM70251572449,Female,27/02/73,30/09/16,C10177,C,COM0000351,A,4.0,4000.0,...,N,S122,G,1000.0,0.0,0.0,0.0,0.0,7,0.56


In [77]:
final_df.to_csv('results.csv')